# Importing eng_data

In [122]:
import pandas as pd

file_path = 'Imports/Station Data/baps1997/fmfxeng.dat'

variable_lengths = {
    "Mainkey": 16, "Channel": 3, "Service Class": 2, "Country": 2,
    "State": 2, "City": 50, "Frequency": 5, "Call Sign": 12,
    "Latitude Indicator": 1, "Latitude Degrees": 2, "Latitude Minutes": 2, "Latitude Seconds": 2,
    "Longitude Ind.": 1, "Longitude Degrees": 3, "Longitude Minutes": 2, "Longitude Seconds": 2,
    "Domestic Status": 6, "Internat. Status": 6, "Coord. Status": 1, "File No. Prefix": 6,
    "App Ref No (ARN)": 8, "Class": 2, "Border": 1, "Border Dist.": 3,
    "Horizontal ERP": 4, "Horiz. Max ERP": 4, "Horizontal HAAT": 5, "Horiz RCAMSL": 5,
    "Vertical ERP": 4, "Vert. Max ERP": 4, "Vertical HAAT": 5, "Vert RCAMSL": 5,
    "Maximum HAAT": 5, "Beam Tilt": 1, "Directional Ant": 1, "Docket No.": 8,
    "Owner": 50, "Filler (1)": 2, "Cutoff Date": 6, "Filler (2)": 2,
    "Window Open Date": 6, "Antenna Make": 3, "Antenna Model": 14, "Rotate": 5,
    "Filler (3)": 2, "Last Notify Date": 6, "Last Notify Time": 6, "Filler (4)": 2,
    "Last Update Date": 6, "Last Update Time": 6, "Is it 73.215": 1, "Filler (5)": 2,
    "CP Expire Date": 6, "Filler (6)": 2, "Filed Lic Date": 6, "Filler (7)": 2,
    "Filed 307 Date": 6, "Internat. Class": 2, "Horiz. RCAGL": 4, "Vert. RCAGL": 4
}



eng_data = pd.read_fwf(file_path, widths=variable_lengths.values(), names=variable_lengths.keys())

eng_data = eng_data.drop_duplicates()

In [123]:
# Convert Date columns to datetime
for col in eng_data.columns:
    if 'Date' in col:
        converted_col = pd.to_datetime(eng_data[col].copy(), format='%y%m%d')
        eng_data[f'{col} DT'] = converted_col
        eng_data.drop(columns=[col], inplace=True)


In [124]:
# Is there a relationship b/w the last update date and the mainkey?
eng_data[['Last Update Date DT', 'Mainkey']]

count = {'year_count' : 0, 'month_count' : 0, 'day_count' : 0}

for index, row in eng_data[['Last Update Date DT', 'Mainkey']].iterrows():

    mainkey = row['Mainkey']
    year = row['Last Update Date DT'].year % 100
    month = row['Last Update Date DT'].month
    day = row['Last Update Date DT'].day

    for time in [(year, 'year_count'), (month, 'month_count'), (day, 'day_count')]:
        if str(time[0]) in mainkey:
            count[time[1]] += 1

print(count)
print(len(eng_data))

# Not really

{'year_count': 2150, 'month_count': 9752, 'day_count': 8495}
25647


In [125]:
# Choose columns to keep
col_names = [
    'Service Class',
    'Country',
    'State',
    'City',
    'Call Sign',
    'Frequency',
    'Domestic Status',
    'File No. Prefix',
    'Last Update Date DT'
]

col_keywords = [
    'Latitude',
    'Longitude',
    'ERP',
    'RCAMSL',
]

cols_to_keep = col_names + [col for col in eng_data.columns if any(keyword in col for keyword in col_keywords)]

assert all([col in eng_data.columns for col in cols_to_keep])

eng_data = eng_data[cols_to_keep]

In [126]:
# Only keep license rows
eng_data = eng_data[eng_data['Domestic Status'] == 'LIC']
eng_data.drop(columns=['Domestic Status'], inplace=True)

In [127]:
# Converting latitude and longitude to decimal
def to_decimal(degrees, minutes, seconds, direction):
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal


eng_data['Latitude Decimal'] = eng_data.apply(lambda row: to_decimal(row['Latitude Degrees'], row['Latitude Minutes'], row['Latitude Seconds'], row['Latitude Indicator']), axis=1)
eng_data['Longitude Decimal'] = eng_data.apply(lambda row: to_decimal(row['Longitude Degrees'], row['Longitude Minutes'], row['Longitude Seconds'], row['Longitude Ind.']), axis=1)

lat_lon_cols_to_drop = [col for col in eng_data.columns if 'Latitude' in col or 'Longitude' in col]
lat_lon_cols_to_drop = [col for col in lat_lon_cols_to_drop if 'Decimal' not in col]
eng_data = eng_data.drop(columns = lat_lon_cols_to_drop )

# Round so info can be fed into API
eng_data = eng_data.rename(columns = {'Latitude Decimal': 'Latitude', 'Longitude Decimal' : 'Longitude'})
eng_data[['Latitude', 'Longitude']] = eng_data[['Latitude', 'Longitude']].map(lambda x: round(x, 10))

In [128]:
# Which ERP to enter?
erp_data = eng_data[[col for col in eng_data.columns if 'ERP' in col]]
for col in erp_data.columns:
    # print(col)
    # print(erp_data[col].isna().sum())
    # print(erp_data[col].notna().sum())
    pass

# Horizontal ERP has the most data

# Which RCAMSL to enter?
rcamsl_data = eng_data[[col for col in eng_data.columns if 'RCAMSL' in col]]

## Apparently vertical ERP is important for aviation, not radio stations

In [129]:
eng_data = eng_data.drop(columns = [col for col in eng_data.columns if 'Vert' in col or 'Max' in col])
eng_data.rename(columns = {'Horizontal ERP': 'ERP', 'Horiz RCAMSL': 'RCAMSL'}, inplace=True)

In [130]:
# Keeping relevant columns and ensuring info in rows
assert len(eng_data.query('`Call Sign`.isna() or Frequency.isna()')) == 0
print('cols dropped:', [col for col in eng_data.columns if eng_data[col].isna().all() or eng_data[col].nunique() == 1])
eng_data = eng_data[[col for col in eng_data.columns if eng_data[col].notna().any() and eng_data[col].nunique() > 1]]

cols dropped: ['Country']


In [131]:
antenna_cols = eng_data.iloc[:,7:]
print(f'I will drop rows w missing antenna info: {list(antenna_cols.columns)}')
rows_to_drop = antenna_cols.isna().any(axis=1)
print(f'number of rows dropped: {rows_to_drop.sum()}')
print(f'portion dropped: {rows_to_drop.sum()/len(eng_data):.2%}')
eng_data = eng_data[~rows_to_drop]

I will drop rows w missing antenna info: ['ERP', 'RCAMSL', 'Latitude', 'Longitude']
number of rows dropped: 601
portion dropped: 5.78%


***Dealing w/ duplicate call signs***

In [132]:
eng_data_duplicates = eng_data[eng_data['Call Sign'].duplicated(keep=False)].sort_values(by='Call Sign')
print(f'portion of duplicates: {len(eng_data_duplicates)/len(eng_data):.2%}')

portion of duplicates: 4.37%


File No. Prefix
| Prefix  | Description                                                        |
|---------|--------------------------------------------------------------------|
| BMLH    | Broadcast License Modification for a commercial FM station (High Power) |
| BLH     | Broadcast License for a commercial FM station (High Power)        |
| BMLED   | Broadcast License Modification for an educational FM station (Low Power) |
| BLED    | Broadcast License for an educational FM station (Low Power)       |

In [133]:
eng_data_duplicates.groupby('Call Sign')['File No. Prefix'].apply(lambda x: set(x)).value_counts()

File No. Prefix
{BLH}            136
{BLH, BMLH}       60
{BLED}             8
{BMLED, BLED}      6
{BMLED}            1
Name: count, dtype: int64

Service Class
| Code | Description                                                                                   |
|------|-----------------------------------------------------------------------------------------------|
| FA   | A vacant or used channel in the Table of Allotments.                                          |
| FM   | A full-service FM station or application.                                                     |
| FS   | A full-service FM station auxiliary transmitting antenna or application.                      |
| FX   | A translator or application for a translator.                                                 |
| FR   | A proposed rule making to amend the Table of Allotments.                                      |
| FB   | A booster station or application for a booster.                                               |

In [134]:
eng_data_duplicates.groupby('Call Sign')['Service Class'].apply(lambda x: set(x)).value_counts()

Service Class
{FM, FS}    203
{FM}          8
Name: count, dtype: int64

In [135]:
indices_to_drop_list = []
indices_to_keep_list = []
indices_processed = []


def process_double_rows(df):
    assert len(df) == 2

    if df['Last Update Date DT'].nunique() == 1 and df['File No. Prefix'].nunique() > 1: # if the dates are the same and file type diff
        mask = df['File No. Prefix'].str.contains('M')
        indices_to_drop = df[mask].index
        indices_to_keep = df[~mask].index
        assert len(indices_to_drop) == 1 and len(indices_to_keep) == 1
        indices_to_drop_list.extend(indices_to_drop)
        indices_to_keep_list.extend(indices_to_keep)
        indices_processed.extend(df.index)


    elif df['Last Update Date DT'].nunique() > 1: # if dates are different and file type is the same
        mask = df['Last Update Date DT'] == df['Last Update Date DT'].max()
        indices_to_keep = df[mask].index
        indices_to_drop = df[~mask].index
        assert len(indices_to_drop) == 1 and len(indices_to_keep) == 1
        indices_to_drop_list.extend(indices_to_drop)
        indices_to_keep_list.extend(indices_to_keep)
        indices_processed.extend(df.index)
    
    else:
        assert 1 == 2


for name, df in eng_data_duplicates.groupby('Call Sign'):

    if set(df['Service Class']) == {'FM'}: # FM service class only
        process_double_rows(df)

    elif set(df['Service Class']) == {'FM', 'FS'}:

        if len(df) == 2:
            indices_to_keep_list.extend(df.index)
            indices_processed.extend(df.index)

        if len(df) == 3:

            assert sorted(df['Service Class']) == ['FM', 'FS', 'FS']
            
            fm_df = df.query("`Service Class` == 'FM'")
            indices_to_keep_list.extend(fm_df.index)
            indices_processed.extend(fm_df.index)


            fs_df = df.query("`Service Class` == 'FS'")


            if (fs_df[['File No. Prefix', 'Last Update Date DT']].nunique() == 1).all(): # If FS rows are the same
                indices_to_keep_list.extend(fs_df.index)
                indices_processed.extend(fs_df.index)
            else:
                process_double_rows(fs_df)

        if len(df) > 3 or len(df) == 1:
            assert 1 == 2 # ensure all cases are covered

    else:
        assert 1==2


assert sorted(indices_processed) == sorted(eng_data_duplicates.index.tolist())
assert len(indices_to_drop_list) + len(indices_to_keep_list) == len(indices_processed) == len(eng_data_duplicates)
assert pd.Series(indices_to_drop_list).isin(indices_to_keep_list).sum() == 0

eng_data = eng_data.drop(index=indices_to_drop_list).copy()
eng_data = eng_data.drop(columns=['Service Class', 'File No. Prefix', 'Last Update Date DT'])

Now, all rows with duplicate call signs in eng_data are refer to multiple antennas broadcasting the same signal

***Is the relationship b/w call sign and (frequency, city, state) bijective?***

In [136]:
eng_data['Tuple'] = eng_data.apply(lambda row: tuple(row[['Frequency', 'City', 'State']]), axis=1)

In [137]:
# How many call signs are there per tuple?
eng_data['numb_uniq_call_signs_per_tuple'] = eng_data.groupby('Tuple')['Call Sign'].transform('nunique')
eng_data['tuple_group_size'] = eng_data.groupby('Tuple')['Call Sign'].transform('size')
eng_data['numb_uniq_call_signs_per_tuple'].value_counts()

numb_uniq_call_signs_per_tuple
1    9739
2      45
Name: count, dtype: int64

In [138]:
# How many call signs are there per tuple?
eng_data['numb_uniq_tuples_per_callsign'] = eng_data.groupby('Call Sign')['Tuple'].transform('nunique')
eng_data['callsign_group_size'] = eng_data.groupby('Call Sign')['Tuple'].transform('size')
eng_data['numb_uniq_tuples_per_callsign'].value_counts()

numb_uniq_tuples_per_callsign
1    9784
Name: count, dtype: int64

Every call sign has a unique tuple.

Not every tuple has a unique call sign.

In [139]:
dup_call_sign_per_tup = eng_data.query('numb_uniq_call_signs_per_tuple > 1').sort_values('Tuple')
# cols_w_unique_info = (dup_call_sign_per_tup.groupby('Tuple').apply(lambda x: x.nunique(), include_groups=False) > 1).any(axis=0)
# dup_call_sign_per_tup = dup_call_sign_per_tup[['Tuple'] + [col for col in cols_w_unique_info.index if cols_w_unique_info[col]]]
print(f'Below are the {len(dup_call_sign_per_tup)} rows of tuples that have multiple call signs:')
dup_call_sign_per_tup.head()

Below are the 45 rows of tuples that have multiple call signs:


,State,City,Call Sign,Frequency,ERP,RCAMSL,Latitude,Longitude,Tuple,numb_uniq_call_signs_per_tuple,tuple_group_size,numb_uniq_tuples_per_callsign,callsign_group_size
166,IL,Chicago,WSSD,88.1,0.01,209.0,41.728889,-87.550833,"(88.1, Chicago, IL)",2,2,1,1
330,IL,Chicago,WCRX,88.1,0.10,226.0,41.872778,-87.647778,"(88.1, Chicago, IL)",2,2,1,1
47,TN,Jackson,W201AP,88.1,0.06,167.0,35.660556,-88.858333,"(88.1, Jackson, TN)",2,2,1,1
172,TN,Jackson,WAMP,88.1,0.75,166.0,35.660556,-88.858333,"(88.1, Jackson, TN)",2,2,1,1
400,IL,Chicago,WZRD,88.3,0.10,209.0,41.982222,-87.718611,"(88.3, Chicago, IL)",2,2,1,1


In [140]:
nuniq_call_sign = eng_data['Call Sign'].nunique()
nuniq_tuple = eng_data['Tuple'].nunique()
print(f'Number of unique (Call Sign, tuple): {nuniq_call_sign, nuniq_tuple}')
print(f'difference = {nuniq_call_sign - nuniq_tuple}')

Number of unique (Call Sign, tuple): (9577, 9555)
difference = 22


It is almost bijective.

The mapping is bijective. Except for 22 additional call signs that map onto the same tuples

In [141]:
cols_to_drop = [col for col in eng_data.columns if 'numb_uniq' in col or 'group_size' in col]
print(cols_to_drop)
eng_data.drop(columns=cols_to_drop, inplace=True)

['numb_uniq_call_signs_per_tuple', 'tuple_group_size', 'numb_uniq_tuples_per_callsign', 'callsign_group_size']


# Match stations in station_data and eng_data

In [142]:
# Import StationData
station_data_full = pd.read_csv('Exports/Data/2.StationData.csv', index_col=0)
station_data = station_data_full.query('broadcast == "FM" and year == 1998 and market != "Puerto Rico" ').copy()

/var/folders/v8/b7w7kd2d6b1_2h62t5lmdqdm0000gn/T/ipykernel_32504/1001790194.py:2: DtypeWarning: Columns (16,18,20,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  station_data_full = pd.read_csv('Exports/Data/2.StationData.csv', index_col=0)


In [143]:
station_data.columns = [f'stat_{col.lower().replace(" ", "_")}' for col in station_data.columns]
eng_data.columns = [f'eng_{col.lower().replace(" ", "_")}' for col in eng_data.columns]

# Keeping relevant columns
assert len(station_data.query('stat_letters.isna() or stat_frequency.isna()')) == 0
cols_to_drop = [col for col in station_data.columns if station_data[col].isna().all() or station_data[col].nunique() == 1]
print('cols dropped:', cols_to_drop)
station_data = station_data[[col for col in station_data.columns if col not in cols_to_drop]].copy()

cols dropped: ['stat_broadcast', 'stat_year', 'stat_homestate', 'stat_hometown', 'stat_hours1', 'stat_hours2', 'stat_hours3', 'stat_format4', 'stat_hours4', 'stat_format5', 'stat_hours5', 'stat_format6', 'stat_hours6', 'stat_format7', 'stat_hours7', 'stat_format8', 'stat_note']


In [144]:
len(station_data), len(eng_data)

(5617, 9784)

I will assume that station_data is the source of truth of 1997 stations

***How to identify stations in eng_data?***

In [145]:
for cols_to_keep in [['eng_call_sign'], ['eng_call_sign', 'eng_frequency'], ['eng_city', 'eng_state', 'eng_frequency'], ['eng_frequency','eng_latitude', 'eng_longitude']]:
    eng_view1 = eng_data[cols_to_keep]
    eng_view1 = eng_view1[eng_view1.duplicated(keep='first')].sort_values(by=cols_to_keep)
    print(f'proportion of duplicate {cols_to_keep}: {len(eng_view1) / len(eng_data):.2%}')

proportion of duplicate ['eng_call_sign']: 2.12%
proportion of duplicate ['eng_call_sign', 'eng_frequency']: 2.12%
proportion of duplicate ['eng_city', 'eng_state', 'eng_frequency']: 2.34%
proportion of duplicate ['eng_frequency', 'eng_latitude', 'eng_longitude']: 1.13%


In [146]:
# Each call sign has a unique frequency
assert (eng_data.groupby('eng_call_sign')['eng_frequency'].apply(lambda x: x.nunique()) == 1).all()

***How to identify stations in station data?***

In [147]:
for cols_to_keep in [['stat_biastationcode'], ['stat_letters']]:
    station_view1 = station_data[cols_to_keep]
    station_view1 = station_view1[station_view1.duplicated(keep='first')].sort_values(by=cols_to_keep)
    print(f'proportion of duplicate {cols_to_keep}: {len(station_view1) / len(eng_data)}')

proportion of duplicate ['stat_biastationcode']: 0.0
proportion of duplicate ['stat_letters']: 0.0


In [148]:
cols_to_keep = ['stat_frequency', 'stat_market']
station_view2 = station_data[cols_to_keep]
station_view2 = station_view2[station_view2['stat_market'] != "Not Applicable"]
station_view2['num_duplicates'] = station_view2.groupby(cols_to_keep)['stat_frequency'].transform('size')
station_view2 = station_view2[station_view2['num_duplicates'] > 1]
duplicate_proportion = len(station_view2) / len(station_data)
print(f'Proportion of duplicate {cols_to_keep} (excluding "Not Applicable"): {duplicate_proportion:.2%}')

Proportion of duplicate ['stat_frequency', 'stat_market'] (excluding "Not Applicable"): 0.62%


In [149]:
# Each call sign has a unique frequency
assert (station_data.groupby('stat_letters')['stat_frequency'].apply(lambda x: x.nunique()) == 1).all()

***First, matching based on (letters, frequency)***

In [150]:
eng_data['eng_call_sign'] = eng_data['eng_call_sign'].str.strip()
station_data['stat_letters'] = station_data['stat_letters'].str.strip()

In [151]:
assert station_data['stat_letters'].duplicated().sum() == 0

In [152]:
before_count = eng_data['eng_call_sign'].duplicated().sum()

def strip_FM(string):
    if len(string) in [5, 6] and string.endswith('FM'):
        return string[:-2]
    return string

eng_data.loc[:,'eng_call_sign'] = eng_data['eng_call_sign'].apply(strip_FM)

assert eng_data['eng_call_sign'].duplicated().sum() == before_count

In [153]:
matching_series = station_data['stat_letters'].isin(eng_data['eng_call_sign'])
print(f'Share of call signs in station data also in eng data: {matching_series.mean():.2%}')

eng_data['call_sign_freq'] = tuple(zip(eng_data['eng_call_sign'], eng_data['eng_frequency']))
station_data['call_sign_freq'] = tuple(zip(station_data['stat_letters'], station_data['stat_frequency']))

matching_series = station_data['call_sign_freq'].isin(eng_data['call_sign_freq'])
print(f'Share of (call signs, frequency) pairs also in eng data: {matching_series.mean():.2%}')

Share of call signs in station data also in eng data: 84.44%
Share of (call signs, frequency) pairs also in eng data: 78.44%


In [154]:
# Identifying state in station_data
assert (station_data['stat_market'].apply(type) == str).all()
station_data['stat_market_state'] = station_data['stat_market'].str.findall(r'([A-Z]{2})')
station_data['stat_market_name'] = station_data['stat_market'].str.replace(r'[A-Z]{2}', '', regex=True).str.strip()
assert (station_data[['stat_market_state','stat_market_name']].notna()).all().all()

In [155]:
# Splitting df
station_data_match_df = station_data[station_data['call_sign_freq'].isin(eng_data['call_sign_freq'])].copy()
station_data_match_df = station_data_match_df.merge(eng_data, left_on='call_sign_freq', right_on='call_sign_freq', how='left', validate='1:m')

station_data_no_match_df = station_data[~station_data['call_sign_freq'].isin(eng_data['call_sign_freq'])].copy()

eng_data_match_df = eng_data[eng_data['call_sign_freq'].isin(station_data['call_sign_freq'])].copy()
eng_data_no_match_df = eng_data[~eng_data['call_sign_freq'].isin(station_data['call_sign_freq'])].copy()

In [156]:
print(f'Number of rows in station_data_match_df: {len(station_data_match_df)}')
print(f'Number of rows in station_data_no_match_df: {len(station_data_no_match_df)}')
print(f'Number of rows in eng_data_match_df: {len(eng_data_match_df)}')
print(f'Number of rows in eng_data_no_match_df: {len(eng_data_no_match_df)}')

Number of rows in station_data_match_df: 4551
Number of rows in station_data_no_match_df: 1211
Number of rows in eng_data_match_df: 4551
Number of rows in eng_data_no_match_df: 5233


In [157]:
def state_match_fn(row):

    if row['stat_market_name'] == 'Not Applicable':
        return True

    elif row['eng_state'] in row['stat_market_state']:
        return True

    return False

station_data_match_df['state_match'] = station_data_match_df.apply(state_match_fn, axis=1)

print('(Call Sign, Frequency) matches with mismatched state')
print(f'Portion of mismatches: {1 - station_data_match_df["state_match"].mean():.2%}')

## View the mismatches:
# print()
# for _, row in station_data_match_df.query("state_match == False")[['stat_market_name', 'stat_market_state', 'eng_city', 'eng_state']].iterrows():
#     print(list(row))

(Call Sign, Frequency) matches with mismatched state
Portion of mismatches: 2.97%


***Second, matching based on (state, city, frequency) in eng_data and (frequency, market) in station_data***

In [158]:
station_data_no_match_w_mark_df = station_data_no_match_df[station_data_no_match_df['stat_market'] != "Not Applicable"].copy()
print(f'number with market: {len(station_data_no_match_w_mark_df)}')
station_data_no_match_wout_mark_df = station_data_no_match_df[station_data_no_match_df['stat_market'] == "Not Applicable"].copy()
print(f'number without market: {len(station_data_no_match_wout_mark_df)}')

number with market: 787
number without market: 424


I will handle the case of unique (state, city, frequency) and (frequency, market). That is, I will exclude rows with duplicate (state, city, frequency) and (frequency, market) from the data sets.

In [159]:
station_data_uniq = station_data_no_match_w_mark_df.copy()
assert station_data_uniq['stat_letters'].duplicated().sum() == 0
station_data_uniq['size'] = station_data_uniq.groupby(['stat_frequency', 'stat_market']).transform('size')
print(f'number of duplicates: {len(station_data_uniq.query("size > 1"))}')
print(f'percent dropped {len(station_data_uniq.query("size > 1")) / len(station_data_uniq):.2%}')
station_data_uniq = station_data_uniq.query('size == 1').drop(columns='size').copy()

number of duplicates: 8
percent dropped 1.02%


In [160]:
eng_data_uniq = eng_data_no_match_df.copy()

eng_data_uniq['size_freq_loc'] = eng_data_uniq.groupby('eng_tuple').transform('size')
eng_data_uniq['ID_freq_loc'] = eng_data_uniq.groupby('eng_tuple').ngroup()
eng_data_uniq['size_call_sign'] = eng_data_uniq.groupby('eng_call_sign').transform('size')
eng_data_uniq['ID_call_sign'] = eng_data_uniq.groupby('eng_call_sign').ngroup()

print(f'Number of eng_tuple pairs that map onto same call sign that will be dropped: {len(eng_data_uniq.query("size_freq_loc != size_call_sign"))}')
print(f'percent dropped {len(eng_data_uniq.query("size_freq_loc != size_call_sign")) / len(eng_data_uniq):.2%}')
eng_data_uniq = eng_data_uniq.query('size_freq_loc == size_call_sign').drop(columns=['ID_freq_loc', 'ID_call_sign', 'size_freq_loc', 'call_sign_freq']).rename(columns={'size_call_sign': 'size'}).copy()

Number of eng_tuple pairs that map onto same call sign that will be dropped: 36
percent dropped 0.69%


My goal is to link all rows in station_data_uniq (which represent unique stations) to a station in eng_data_uniq (which are identified by the freq, location tuple)

In [161]:
# First, I will drop rows from station_data without a frequency match
before_drop = len(station_data_uniq)

station_data_uniq = station_data_uniq.query('stat_frequency in @eng_data_uniq["eng_frequency"]').copy()
print(f'Portion of rows w/out frequency match dropped: {(before_drop - len(station_data_uniq))/before_drop :.2%}')

assert (station_data_uniq['stat_frequency'].isin(eng_data_uniq['eng_frequency'])).all()

Portion of rows w/out frequency match dropped: 0.13%


Now every row in station_data_uniq has a corresponding frequency in eng_data_uniq

In [162]:
from itertools import chain
all_states = set(chain.from_iterable(station_data_uniq['stat_market_state']))
all_frequencies = set(station_data_uniq['stat_frequency'])

In [163]:
dfs_to_match = {}

# gather all possible (frequency, state) pairs
for frequency in all_frequencies:
    station_view = station_data_uniq.query('stat_frequency == @frequency')[['stat_market', 'stat_market_name', 'stat_market_state']]
    eng_view = eng_data_uniq.query('eng_frequency == @frequency')[['eng_city', 'eng_state', 'size']]

    for state in all_states:
        station_markets = station_view[station_view['stat_market_state'].apply(lambda x: state in x)][['stat_market', 'stat_market_name']].copy()    
        eng_cities = eng_view.query('eng_state == @state')[['eng_city', 'size']].copy()

        if not station_markets.empty:
            dfs_to_match[(frequency, state)] = (eng_cities, station_markets)

            
assert sum([len(v[1]) for v in dfs_to_match.values()]) == len(station_data_uniq) + len(station_data_uniq[station_data_uniq['stat_market_state'].apply(len) == 2])

I need to consider the cases where a single station has multiple antennas

In [164]:
for k, v in dfs_to_match.items():
    if (2 == v[0]['size']).any():
        print(k)
        print(v[0].head())
        print()

(104.3, 'CA')
            eng_city  size
20557    Los Angeles     2
20595  Carmel Valley     1
20627    Los Angeles     2
20653       Oakhurst     1
20688          Davis     1

(105.5, 'AL')
            eng_city  size
22167  Muscle Shoals     2
22393  Muscle Shoals     2

(105.5, 'GA')
        eng_city  size
22350  Rossville     2
22453  Rossville     2

(105.1, 'NY')
       eng_city  size
21739  Fairport     1
21770  New York     2
21867  New York     2

(107.3, 'NE')
      eng_city  size
24742  Lincoln     2
24811  Lincoln     2

(100.3, 'CA')
          eng_city  size
15466  Los Angeles     1
15483     San Jose     2
15525     San Jose     2
15557  Bakersfield     1

(92.3, 'KS')
     eng_city  size
5461   Newton     2
5473   Newton     2

(97.5, 'NY')
            eng_city  size
12039      Patchogue     2
12066  Hoosick Falls     1
12100      Patchogue     2

(99.1, 'TX')
      eng_city  size
13882  Houston     2
13949  Houston     2
13990    Tyler     1
14058   Odessa     1

(102.9,

In [165]:
from fuzzywuzzy import fuzz
from scipy.optimize import linear_sum_assignment
import numpy as np

def find_closest_match(market_df, city_df, threshold=70):
    assert not market_df.empty

    market_df = market_df.reset_index().rename(columns={'index': 'indices'})

    maket_name_full_series = market_df.loc[:,'stat_market']
    market_name_cut_series = market_df.loc[:,'stat_market_name']
    market_index_series = market_df.loc[:,'indices'].astype(int)

    city_df_size2 = (
    city_df.query('size == 2')
    .assign(indices=lambda df: df.index.map(int))
    .groupby('eng_city', as_index=False)
    .agg({'size': 'first', 'indices': lambda x: tuple(x)})
    )
    city_df_size1 = city_df.query('size == 1').reset_index().rename(columns={'index': 'indices'})
    city_df_size1.indices = city_df_size1.indices.astype(int)
    city_df_collapsed = pd.concat([city_df_size2, city_df_size1]).reset_index(drop=True)
    city_df_collapsed.drop(columns='size', inplace=True)

    city_name_series = city_df_collapsed.loc[:,'eng_city']
    city_index_series = city_df_collapsed.loc[:,'indices']

    if city_df.empty:
        return {
            (maket_name_full_series.iloc[i], market_index_series.iloc[i]): {
                "matched_entry": None,
                "reason": "no cities to match with"
            }
            for i in range(len(maket_name_full_series))
        }

    if len(market_df) == 1 and len(city_df) == 1:
        return {
            (maket_name_full_series.iloc[0], market_index_series.iloc[0]): {
                "matched_entry": (city_name_series.iloc[0], city_index_series.iloc[0]),
                "reason": 'one_to_one',
            }
        }

    score_matrix = np.zeros((len(market_df), len(city_df)))

    for i, market_name in enumerate(market_name_cut_series):
        for j, city_name in enumerate(city_name_series):
            score_matrix[i, j] = fuzz.ratio(market_name, city_name)

    score_matrix[score_matrix < threshold] = 0

    row_indices, col_indices = linear_sum_assignment(-score_matrix)

    matches_within_series = {}

    for row, col in zip(row_indices, col_indices):
        market_name, market_index = maket_name_full_series.iloc[row], market_index_series.iloc[row]
        city_name, city_index = city_name_series.iloc[col], city_index_series.iloc[col]
        score = score_matrix[row, col]

        if score >= threshold:
            matches_within_series[(market_name, market_index)] = {
                "matched_entry": (city_name, city_index),
                "reason": score
            }
        else:
            matches_within_series[(market_name, market_index)] = {
                "matched_entry": None,
                "reason": "score lower than threshold"
            }

    unmatched_rows = set(range(len(market_df))) - set(row_indices)

    for row in unmatched_rows:
        market_name, market_index = maket_name_full_series.iloc[row], maket_name_full_series.index[row]
        matches_within_series[(market_name, market_index)] = { 
            "matched_entry": None,
            "reason": "no more cities to match with (markets < cities)"
        }

    return matches_within_series

In [166]:
matches_dict = {}

for key, value in dfs_to_match.items():
    assert len(value) == 2
    (eng_cities_df, station_market_df) = value
    assert type(eng_cities_df) == pd.DataFrame and type(station_market_df) == pd.DataFrame

    matches_with_scores = find_closest_match(station_market_df, eng_cities_df)
    matches_dict[key] = matches_with_scores

In [167]:
# Create a list of tuples to match
city_market_matches = []
for key, value in matches_dict.items():
    for (market_name, market_index), matched_entry_dict in value.items():
        if matched_entry_dict['matched_entry'] is None:
            continue
        (city_name, city_index) = matched_entry_dict['matched_entry']
        reason = matched_entry_dict['reason']
        city_market_matches.append((key, market_name, market_index, city_name, city_index, reason))

In [168]:
city_market_matches_df = pd.DataFrame(city_market_matches, columns=['key','market', 'market_index', 'city', 'city_index', 'match_reason'])
city_market_matches_df.head()

,key,market,market_index,city,city_index,match_reason
0,"(102.3, NC)",Fayetteville NC,128508,Lumberton,17922,one_to_one
1,"(102.3, CT)",New London CT,99533,Stonington,18019,one_to_one
2,"(102.3, NH)",Manchester NH,182829,Concord,18010,one_to_one
3,"(102.3, IA)",Dubuque IA,86543,Dubuque,18228,100.0
4,"(102.3, IN)",Ft. Wayne IN,110905,Auburn,17898,one_to_one


In [169]:
assert city_market_matches_df.duplicated(subset=['city_index']).sum() == 0

# Drop rows of markets in two states that have perfect matches in both states
multiple_matches = city_market_matches_df[city_market_matches_df.duplicated(subset=['market_index'], keep=False)].sort_values(by='market_index')
assert multiple_matches['match_reason'].eq('one_to_one').all()
city_market_matches_df = city_market_matches_df[city_market_matches_df.index.isin(multiple_matches.index) == False].copy()

assert city_market_matches_df.duplicated(subset=['market_index']).sum() == 0

multiple_matches.head()

,key,market,market_index,city,city_index,match_reason
331,"(101.5, IA)",Omaha-Council Bluffs NE-IA,33695,Decorah,17060,one_to_one
332,"(101.5, NE)",Omaha-Council Bluffs NE-IA,33695,Hastings,16973,one_to_one
81,"(101.3, IA)",Quad Cities (Davenport-Rock Island-Moline IA-IL),78829,Creston,16681,one_to_one
83,"(101.3, IL)",Quad Cities (Davenport-Rock Island-Moline IA-IL),78829,East Moline,16772,one_to_one
137,"(103.3, VT)",Burlington-Plattsburgh VT-NY,163381,Waterbury,19445,one_to_one


In [170]:
print('tuple rows to expand:')
city_market_matches_df[city_market_matches_df['city_index'].apply(lambda x: isinstance(x, tuple))].head()

tuple rows to expand:


,key,market,market_index,city,city_index,match_reason
35,"(105.5, AL)",Florence-Muscle Shoals AL,214751,Muscle Shoals,"(22167, 22393)",74.0
40,"(105.1, NY)",New York NY,207593,New York,"(21770, 21867)",100.0
69,"(107.3, NE)",Lincoln NE,5476,Lincoln,"(24742, 24811)",100.0
122,"(102.9, IA)",Cedar Rapids IA,90926,Cedar Rapids,"(18733, 18922)",100.0
125,"(102.9, CA)",San Diego CA,43606,San Diego,"(18717, 18772)",100.0


In [171]:
# expand tuples
city_market_matches_df = city_market_matches_df.explode('city_index').reset_index(drop=True)
assert city_market_matches_df['city_index'].apply(lambda x: isinstance(x, (np.int64, int))).all()

In [172]:
city_market_matches_df = city_market_matches_df.merge(station_data_uniq, left_on=['market_index'], right_index=True, how='left', validate='m:1')

city_market_matches_df = city_market_matches_df.merge(eng_data_uniq, left_on=['city_index'], right_index=True, how='left', validate='1:1')
city_market_matches_df.drop(columns=['market', 'market_index', 'city', 'city_index', 'eng_tuple', 'key'], inplace=True)

cols_to_front = ['stat_market_name', 'eng_city', 'stat_market_state', 'eng_state', 'stat_frequency', 'eng_frequency']
city_market_matches_df = city_market_matches_df[cols_to_front + [col for col in city_market_matches_df.columns if col not in cols_to_front]]

In [173]:
assert (city_market_matches_df['stat_frequency'] == city_market_matches_df['eng_frequency']).all()
for row in city_market_matches_df.itertuples():
    assert row.eng_state in row.stat_market_state 

In [174]:
new_matches_column_names = set(city_market_matches_df.columns)
old_matches_column_names = set(station_data_match_df.columns)
common_columns = sorted(list(new_matches_column_names.intersection(old_matches_column_names))) + ['match_reason']
print(new_matches_column_names.difference(old_matches_column_names))
print(old_matches_column_names.difference(new_matches_column_names))

{'size', 'match_reason'}
{'eng_tuple', 'state_match'}


In [175]:
# Filter both DataFrames to include only the common columns
df1 = city_market_matches_df[common_columns]
station_data_match_df['match_reason'] = 'letter_match'
df2 = station_data_match_df[common_columns]

# Concatenate the DataFrames along rows
matched_data = pd.concat([df1, df2], axis=0)
matched_data.drop(columns='call_sign_freq', inplace=True)

In [176]:
matched_data.head()

,eng_call_sign,eng_city,eng_erp,eng_frequency,eng_latitude,eng_longitude,eng_rcamsl,eng_state,stat_biamarketid,stat_biastationcode,...,stat_frequency,stat_letters,stat_market,stat_market_name,stat_market_state,stat_rank,stat_rating,stat_startdate,stat_supermarket,match_reason
0,WJSK,Lumberton,3.00,102.3,34.599444,-79.009167,122.0,NC,136.0,15938.0,...,102.3,WFNC,Fayetteville NC,Fayetteville,[NC],129.0,0.50,1964.0,NaN,one_to_one
1,WVVE,Stonington,3.00,102.3,41.406389,-71.837500,137.0,CT,262.0,17132.0,...,102.3,WAXK,New London CT,New London,[CT],171.0,4.15,1981.0,NaN,one_to_one
2,WKXL,Concord,3.00,102.3,43.216667,-71.576111,222.0,NH,159.0,16296.0,...,102.3,WOTX,Manchester NH,Manchester,[NH],187.0,NaN,1972.0,Portsmouth-Rochester-Dover NH-ME,one_to_one
3,KGGY,Dubuque,1.65,102.3,42.541111,-90.612778,374.0,IA,200.0,11903.0,...,102.3,KXGE,Dubuque IA,Dubuque,[IA],225.0,5.15,1980.0,NaN,100.0
4,WGL,Auburn,3.00,102.3,41.333611,-85.052222,355.0,IN,108.0,15601.0,...,102.3,WCKZ,Ft. Wayne IN,Ft. Wayne,[IN],103.0,0.20,1967.0,NaN,one_to_one


In [177]:
matched_data.columns

Index(['eng_call_sign', 'eng_city', 'eng_erp', 'eng_frequency', 'eng_latitude',
       'eng_longitude', 'eng_rcamsl', 'eng_state', 'stat_biamarketid',
       'stat_biastationcode', 'stat_format1', 'stat_format2', 'stat_format3',
       'stat_frequency', 'stat_letters', 'stat_market', 'stat_market_name',
       'stat_market_state', 'stat_rank', 'stat_rating', 'stat_startdate',
       'stat_supermarket', 'match_reason'],
      dtype='object')

In [178]:
matched_data.match_reason.value_counts()

match_reason
letter_match    4551
one_to_one       289
100.0             59
74.0               3
71.0               3
79.0               1
80.0               1
90.0               1
72.0               1
92.0               1
Name: count, dtype: int64

In [179]:
print(f'rows successfully matched: {len(matched_data)/len(station_data):.2%}')

rows successfully matched: 87.41%


In [180]:
matched_data.to_csv('Exports/Data/3.1997StationDatawAntennaInfo.csv', index=False)